# Pseudobulk differential expression to compare gtl1 vs WT

In [1]:
library(tidyverse)
library(Seurat)
library(cowplot)
library(ComplexHeatmap)
library(circlize)
library(GeneOverlap)
library(gprofiler2)
library(ggrepel)
library(muscat)
library(purrr)
library(limma)
library(scran)
library(future)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Registered S3 method overwritten by 'spatstat.geom':
  method     from
  print.boxx cli 

Attaching SeuratObject

Loading required package: grid

ComplexHeatmap version 2.11.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export static 
complex hea

In [2]:
#for 200gb ram 
options(future.globals.maxSize = 200000 * 1024^2)

In [3]:
rc.integrated <- readRDS("../data/integrations/rc.integrated_11S_gtl1_df1_Li_seu3_clean.rds")

In [4]:
rc.integrated

An object of class Seurat 
70780 features across 74810 samples within 3 assays 
Active assay: integrated (17681 features, 17681 variable features)
 2 other assays present: RNA, SCT
 4 dimensional reductions calculated: pca, umap, umap_3D, umap_2D

## Cell type and developmental stage metadata

- Developmental stage: `time_zone`
- Cell type:`cell_type`
- Combination of cell type and developmental stage: `time_zone_cell_type`
- Combination of cell type and developmental stage with cell subtypes (not used): `time_zone_cell_subtypes` 

In [5]:
feature_names <- read_tsv("./data/features.tsv.gz", col_names = c("AGI", "Name", "Type")) %>%
  select(-Type) %>%
  distinct()


── Column specification ────────────────────────────────────────────────────────
cols(
  AGI = col_character(),
  Name = col_character(),
  Type = col_character()
)




In [6]:
table(rc.integrated$genotype)


     df1     gtl1 gtl1_df1       WT 
   15678    22594    17634    18904 

In [7]:
rc.integrated <- subset(rc.integrated, 
                        subset = sample %in% c("sc_122",
                                              "sc_123",
                                              "sc_124",
                                              "sc_125",
                                              "sc_126",
                                              "sc_127",
                                              "sc_128",
                                              "sc_129"))

In [8]:
rc.integrated$genotype <- factor(rc.integrated$genotype, 
                                 levels=c("WT", 
                                          "gtl1", 
                                          "df1", 
                                          "gtl1_df1"))

In [9]:
table(rc.integrated$genotype)


      WT     gtl1      df1 gtl1_df1 
   18904    22594    15678    17634 

# Differential State Analysis with Muscat

In [10]:
# subset samples you want to compare

integrated.de <- subset(rc.integrated, subset = sample %in% c("sc_122", "sc_126","sc_123","sc_127"))
integrated.de$genotype <- factor(integrated.de$genotype, levels=c("WT", "gtl1"))

In [11]:
integrated.de

An object of class Seurat 
70780 features across 41498 samples within 3 assays 
Active assay: integrated (17681 features, 17681 variable features)
 2 other assays present: RNA, SCT
 4 dimensional reductions calculated: pca, umap, umap_3D, umap_2D

## Convert to sce

In [12]:
#  construct sce manually
my_metadata <- data.frame(sample_id = integrated.de$sample,
                              group_id = integrated.de$genotype,
                              cluster_id = integrated.de$time_zone_cell_type, 
                             rep=integrated.de$rep) # include experimental rep as co-variate

sce <- SingleCellExperiment(assays = list(counts = integrated.de@assays$RNA@counts),
	                            colData = my_metadata)

In [13]:
    (sce <- prepSCE(sce, 
        kid = "cluster_id", # subpopulation assignments
        gid = "group_id",   # group IDs (ctrl/stim)
        sid = "sample_id",    # sample IDs (ctrl/stim.1234)
        drop = FALSE))        # drop all other colData columns

class: SingleCellExperiment 
dim: 28688 41498 
metadata(1): experiment_info
assays(1): counts
rownames(28688): AT1G01010 AT1G01020 ... AT5G37474 AT5G07835
rowData names(0):
colnames(41498): AAACCCAAGAGCCGTA_4 AAACCCAAGCATTGTC_4 ...
  TTTGTTGGTGTTCGTA_9 TTTGTTGGTTTACCTT_9
colData names(4): cluster_id sample_id group_id rep
reducedDimNames(0):
altExpNames(0):

## pre-filtering

In [14]:
# remove undetected genes
sce <- sce[rowSums(counts(sce) > 0) > 0, ]
dim(sce)

[1] 26374 41498

In [15]:
# remove lowly expressed genes
sce <- sce[rowSums(counts(sce) > 1) >= 1, ]
dim(sce)

[1] 21805 41498

In [16]:
# create pseudobulk profiles
pb <- aggregateData(sce,
    assay = "counts", fun = "sum",
    by = c("cluster_id", "sample_id"))
# one sheet per subpopulation
assayNames(pb)

[1] "Distal Columella"                     
 [2] "Distal Lateral Root Cap"              
 [3] "Elongation_Atrichoblast"              
 [4] "Elongation_Cortex"                    
 [5] "Elongation_Endodermis"                
 [6] "Elongation_Pericycle"                 
 [7] "Elongation_Phloem"                    
 [8] "Elongation_Procambium"                
 [9] "Elongation_Trichoblast"               
[10] "Elongation_Xylem"                     
[11] "Maturation_Atrichoblast"              
[12] "Maturation_Cortex"                    
[13] "Maturation_Endodermis"                
[14] "Maturation_Pericycle"                 
[15] "Maturation_Phloem"                    
[16] "Maturation_Procambium"                
[17] "Maturation_Trichoblast"               
[18] "Maturation_Xylem"                     
[19] "Proliferation Domain_Atrichoblast"    
[20] "Proliferation Domain_Cortex"          
[21] "Proliferation Domain_Endodermis"      
[22] "Proliferation Domain_Pericycle"       
[23] "Proliferation Domain_Phloem"          
[24] "Proliferation Domain_Quiescent Center"
[25] "Proliferation Domain_Trichoblast"     
[26] "Proximal Columella"                   
[27] "Proximal Lateral Root Cap"            
[28] "Transition Domain_Atrichoblast"       
[29] "Transition Domain_Cortex"             
[30] "Transition Domain_Pericycle"          
[31] "Transition Domain_Phloem"             
[32] "Transition Domain_Trichoblast"        
[33] "Transition Domain_Xylem"

In [17]:
# pseudobulks for 1st subpopulation
t(head(assay(pb)))

,AT1G01010,AT1G01020,AT1G01030,AT1G01040,AT1G01050,AT1G01060
sc_122,37,32,12,9,862,6
sc_123,45,48,13,5,1051,9
sc_126,26,41,17,12,1287,9
sc_127,48,31,12,7,857,9


In [18]:
# metadata to manually add to DE contrast
bscs <- read.csv("./data/GEO_upload_BR_time_scRNA_samples_metadata_with_stats.csv", na.strings=c("","NA"), stringsAsFactors = F)
bscs$date <- gsub('^([0-9]{4})([0-9]{2})([0-9]+)$', '\\1-\\2-\\3', bscs$date)

In [19]:
# experiment info for contrasts, add rep from csv

ei <- metadata(sce)$experiment_info
ei

sample_date <- select(bscs, sample_id=sample, rep=rep)

ei <- left_join(ei, sample_date)

ei

sample_id,group_id,n_cells
<fct>,<fct>,<dbl>
sc_122,WT,11614
sc_123,gtl1,11141
sc_126,WT,7290
sc_127,gtl1,11453


Joining, by = "sample_id"



sample_id,group_id,n_cells,rep
<chr>,<fct>,<dbl>,<int>
sc_122,WT,11614,1
sc_123,gtl1,11141,1
sc_126,WT,7290,2
sc_127,gtl1,11453,2


In [20]:
mm <- model.matrix(~ 0 + ei$group_id + ei$rep)
dimnames(mm) <- list(ei$sample_id, c(levels(ei$group_id), "rep"))

mm

,WT,gtl1,rep
sc_122,1,0,1
sc_123,0,1,1
sc_126,1,0,2
sc_127,0,1,2


In [21]:
contrast <- makeContrasts("gtl1-WT", levels = mm)

contrast

,gtl1-WT
WT,-1
gtl1,1
rep,0


In [22]:
res <- pbDS(pb, design = mm, 
            contrast = contrast, 
            method="edgeR", 
            min_cells=5, 
            filter = c("none"))

Distal Columella..Distal Lateral Root Cap..Elongation_Atrichoblast..Elongation_Cortex..Elongation_Endodermis..Elongation_Pericycle..Elongation_Phloem..Elongation_Procambium..Elongation_Trichoblast..Elongation_Xylem..Maturation_Atrichoblast..Maturation_Cortex..Maturation_Endodermis..Maturation_Pericycle..Maturation_Phloem..Maturation_Procambium..Maturation_Trichoblast..Maturation_Xylem..Proliferation Domain_Atrichoblast..Proliferation Domain_Cortex..Proliferation Domain_Endodermis..Proliferation Domain_Pericycle..Proliferation Domain_Phloem..Proliferation Domain_Quiescent Center..Proliferation Domain_Trichoblast..Proximal Columella..Proximal Lateral Root Cap..Transition Domain_Atrichoblast..Transition Domain_Cortex..Transition Domain_Pericycle..Transition Domain_Phloem..Transition Domain_Trichoblast..Transition Domain_Xylem..

### DEG results

In [23]:
# DEG results with gene freqs
(res_to_write_frq <- resDS(sce, res, bind = "row", cpm=TRUE, frq=T))

gene,cluster_id,sc_122.cpm,sc_126.cpm,sc_123.cpm,sc_127.cpm,sc_122.frq,sc_126.frq,sc_123.frq,sc_127.frq,WT.frq,gtl1.frq,logFC,logCPM,F,p_val,p_adj.loc,p_adj.glb,contrast
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
AT1G01010,Distal Columella,11.100,6.520,11.600,14.700,0.06100,0.05040,0.07410,0.08850,0.055900,0.081100,0.59400,3.5100,5.010000,0.0252,0.632,1,gtl1-WT
AT1G01020,Distal Columella,9.580,10.300,12.400,9.490,0.05570,0.07170,0.07940,0.05840,0.063400,0.069200,0.13400,3.4600,0.250000,0.6170,0.999,1,gtl1-WT
AT1G01030,Distal Columella,3.590,4.270,3.350,3.670,0.01970,0.03290,0.02290,0.02070,0.026100,0.021900,-0.15800,2.1000,0.131000,0.7180,0.999,1,gtl1-WT
AT1G01040,Distal Columella,2.690,3.010,1.290,2.140,0.01620,0.01940,0.00882,0.01320,0.017700,0.010900,-0.73400,1.5000,1.510000,0.2190,0.999,1,gtl1-WT
AT1G01050,Distal Columella,258.000,323.000,271.000,262.000,0.69300,0.77900,0.72700,0.67600,0.734000,0.702000,-0.11400,8.1300,0.469000,0.4930,0.999,1,gtl1-WT
AT1G01060,Distal Columella,1.800,2.260,2.320,2.760,0.01080,0.01740,0.01590,0.01510,0.014000,0.015500,0.32100,1.5000,0.297000,0.5860,0.999,1,gtl1-WT
AT1G01070,Distal Columella,0.599,0.251,0.000,0.000,0.00359,0.00194,0.00000,0.00000,0.002800,0.000000,-3.77000,-0.3950,1.390000,0.2390,0.999,1,gtl1-WT
AT1G01080,Distal Columella,0.299,0.251,0.258,0.612,0.00180,0.00194,0.00176,0.00377,0.001860,0.002730,0.60000,-0.1540,0.092300,0.7610,0.999,1,gtl1-WT
AT1G01090,Distal Columella,39.200,41.900,31.500,42.600,0.19600,0.26400,0.17500,0.22800,0.228000,0.200000,-0.14200,5.3000,0.546000,0.4600,0.999,1,gtl1-WT


In [24]:
## all genes background 

all_bg <- filter(res_to_write_frq,
                WT.frq >=0.1 | gtl1.frq >=0.1)

In [25]:
length(unique(all_bg$gene))

[1] 16756

In [26]:
#total DE genes p_adj.loc < 0.05, abs(logFC) > 1.5
sig_DE <- filter(res_to_write_frq, p_adj.loc<=0.05 & abs(logFC) > log2(1.5))
sig_DE <- left_join(sig_DE, feature_names, by=c("gene"="AGI"))

length(unique(sig_DE$gene))

[1] 4385

In [27]:
# filter gene freqs to avoid calling lowly detected genes
sig_DE_fil <- filter(sig_DE, WT.frq >=0.1 | gtl1.frq >=0.1)

In [28]:
length(unique(sig_DE_fil$gene))

[1] 3281

In [29]:
# load TFs
TF_list <- read_csv("./data/Kay_TF_thalemine_annotations.csv", col_names = c("gene", "TF_Name", "Description")) 


── Column specification ────────────────────────────────────────────────────────
cols(
  gene = col_character(),
  TF_Name = col_character(),
  Description = col_character()
)




In [30]:
sig_DE_fil <- left_join(sig_DE_fil, TF_list)

Joining, by = "gene"



In [31]:
# label up vs down
sig_DE_fil <- sig_DE_fil %>%
  mutate(up_dn_label = case_when(logFC >=log2(1.5) ~ "Up",  
                                       logFC <=log2(1/1.5) ~ "Down",
                                       TRUE ~ "Not DE"))

sig_DE_fil$clust_up_dn <- paste(sig_DE_fil$cluster_id, sig_DE_fil$up_dn_label, sep="_")

sig_DE_fil

gene,cluster_id,sc_122.cpm,sc_126.cpm,sc_123.cpm,sc_127.cpm,sc_122.frq,sc_126.frq,sc_123.frq,sc_127.frq,⋯,F,p_val,p_adj.loc,p_adj.glb,contrast,Name,TF_Name,Description,up_dn_label,clust_up_dn
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
AT1G01470,Distal Columella,3150.0,3230.0,1790.00,1650.00,0.9660,0.9260,0.8660,0.8360,⋯,28.7,8.68e-08,4.75e-05,4.55e-02,gtl1-WT,LEA14,NA,NA,Down,Distal Columella_Down
AT1G05680,Distal Columella,735.0,812.0,509.00,497.00,0.5870,0.5500,0.5100,0.4600,⋯,13.9,1.98e-04,3.00e-02,1.00e+00,gtl1-WT,UGT74E2,NA,NA,Down,Distal Columella_Down
AT1G06080,Distal Columella,11.1,15.8,43.10,46.50,0.0503,0.0853,0.1900,0.2000,⋯,66.9,2.97e-16,1.15e-12,1.57e-10,gtl1-WT,ADS1,NA,NA,Up,Distal Columella_Up
AT1G12560,Distal Columella,12.6,21.1,29.40,27.90,0.0610,0.1430,0.1570,0.1470,⋯,13.4,2.55e-04,3.54e-02,1.00e+00,gtl1-WT,EXPA7,NA,NA,Up,Distal Columella_Up
AT1G14960,Distal Columella,48.8,56.7,84.40,79.90,0.2440,0.3430,0.4230,0.3730,⋯,12.8,3.53e-04,4.37e-02,1.00e+00,gtl1-WT,AT1G14960,NA,NA,Up,Distal Columella_Up
AT1G15380,Distal Columella,15.6,52.7,57.30,46.50,0.0664,0.1490,0.1990,0.1730,⋯,13.9,1.95e-04,2.98e-02,1.00e+00,gtl1-WT,AT1G15380,NA,NA,Up,Distal Columella_Up
AT1G19840,Distal Columella,10.5,10.8,23.00,17.80,0.0610,0.0775,0.1360,0.1020,⋯,15.4,8.50e-05,1.50e-02,1.00e+00,gtl1-WT,AT1G19840,NA,NA,Up,Distal Columella_Up
AT1G23720,Distal Columella,41.0,53.7,81.00,82.40,0.2030,0.2930,0.3490,0.3390,⋯,19.3,1.13e-05,2.93e-03,1.00e+00,gtl1-WT,AT1G23720,NA,NA,Up,Distal Columella_Up
AT1G30135,Distal Columella,102.0,39.4,92.10,125.00,0.2550,0.1690,0.2650,0.2710,⋯,16.3,5.47e-05,1.07e-02,1.00e+00,gtl1-WT,TIFY5A,JAZ8,jasmonate-zim-domain protein 8,Up,Distal Columella_Up


In [32]:
sig_DE_fil
write.csv(sig_DE_fil, file = "./output/v4_gtl1_v_WT_cell_time_EdgeR_q0.05_FC1.5_r_v_4_20220121.csv")

gene,cluster_id,sc_122.cpm,sc_126.cpm,sc_123.cpm,sc_127.cpm,sc_122.frq,sc_126.frq,sc_123.frq,sc_127.frq,⋯,F,p_val,p_adj.loc,p_adj.glb,contrast,Name,TF_Name,Description,up_dn_label,clust_up_dn
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
AT1G01470,Distal Columella,3150.0,3230.0,1790.00,1650.00,0.9660,0.9260,0.8660,0.8360,⋯,28.7,8.68e-08,4.75e-05,4.55e-02,gtl1-WT,LEA14,NA,NA,Down,Distal Columella_Down
AT1G05680,Distal Columella,735.0,812.0,509.00,497.00,0.5870,0.5500,0.5100,0.4600,⋯,13.9,1.98e-04,3.00e-02,1.00e+00,gtl1-WT,UGT74E2,NA,NA,Down,Distal Columella_Down
AT1G06080,Distal Columella,11.1,15.8,43.10,46.50,0.0503,0.0853,0.1900,0.2000,⋯,66.9,2.97e-16,1.15e-12,1.57e-10,gtl1-WT,ADS1,NA,NA,Up,Distal Columella_Up
AT1G12560,Distal Columella,12.6,21.1,29.40,27.90,0.0610,0.1430,0.1570,0.1470,⋯,13.4,2.55e-04,3.54e-02,1.00e+00,gtl1-WT,EXPA7,NA,NA,Up,Distal Columella_Up
AT1G14960,Distal Columella,48.8,56.7,84.40,79.90,0.2440,0.3430,0.4230,0.3730,⋯,12.8,3.53e-04,4.37e-02,1.00e+00,gtl1-WT,AT1G14960,NA,NA,Up,Distal Columella_Up
AT1G15380,Distal Columella,15.6,52.7,57.30,46.50,0.0664,0.1490,0.1990,0.1730,⋯,13.9,1.95e-04,2.98e-02,1.00e+00,gtl1-WT,AT1G15380,NA,NA,Up,Distal Columella_Up
AT1G19840,Distal Columella,10.5,10.8,23.00,17.80,0.0610,0.0775,0.1360,0.1020,⋯,15.4,8.50e-05,1.50e-02,1.00e+00,gtl1-WT,AT1G19840,NA,NA,Up,Distal Columella_Up
AT1G23720,Distal Columella,41.0,53.7,81.00,82.40,0.2030,0.2930,0.3490,0.3390,⋯,19.3,1.13e-05,2.93e-03,1.00e+00,gtl1-WT,AT1G23720,NA,NA,Up,Distal Columella_Up
AT1G30135,Distal Columella,102.0,39.4,92.10,125.00,0.2550,0.1690,0.2650,0.2710,⋯,16.3,5.47e-05,1.07e-02,1.00e+00,gtl1-WT,TIFY5A,JAZ8,jasmonate-zim-domain protein 8,Up,Distal Columella_Up


In [33]:
# add DE and up/dn to total list
sig_to_join <- sig_DE_fil %>%
mutate(clust_gene=paste(cluster_id, gene, sep="_")) %>%
select(clust_gene, up_dn_label, clust_up_dn)

In [34]:
# join all genes list to DE labels
all_bg <- mutate(all_bg, clust_gene=paste(cluster_id, gene, sep="_"))

all_bg <- left_join(all_bg, feature_names, by=c("gene"="AGI"))

all_bg$DE <- all_bg$clust_gene %in% sig_to_join$clust_gene


all_bg <- all_bg %>%
left_join(sig_to_join, by="clust_gene") %>%
arrange(all_bg, p_adj.loc)

write.csv(all_bg, file = "./output/v4_all_gtl1_v_WT_cell_time_EdgeR_q0.05_FC1.5_r_v_4_20220121.csv")